# STS with CP - first model draft

## 1. Instance Parameters

The instance parameter of this STS problem is $n$, which refers to the number of teams.
The other variables are derived from $n$:
- number of weeks in the tournament = $n - 1$
- number of periods for each week = $\frac{n}{2}$
- number of slots for each period = $2$ (which means that the team in the first slot plays at home and the team in the second slot plays away)

In [1]:
%%writefile "./first_model_draft.mzn"
include "alldifferent.mzn";

int: n;
int: weeks = n - 1;
int: periods = n div 2;
int: slots = 2;

Overwriting ./first_model_draft.mzn


## 2. Decision Variables

In this first draft just one decision variable was used: $game_{w,p,s}$, where $w = 1, ..., weeks$, $p = 1, ..., periods$ and $s = 1, ..., slots$.
- $game_{w,p,1}$ referes to teams that play at home
- $game_{w,p,2}$ referes to teams that play away

In [2]:
%%writefile -a "./first_model_draft.mzn"

array[1..weeks, 1..periods, 1..slots] of var 1..n: game;

Appending to ./first_model_draft.mzn


## 3. Constraints

These are the explicit constraints written in the requirements of the project:
TODO improve this explanation

**1. every team plays with every other team only once**

Let $game[w,p,s]$ be the team in week $w$, period $p$, slot $s$. Then:

$\forall i,j \in {1,\dots,n},\ i < j: \quad \sum_{w=1}^{\text{weeks}} \sum_{p=1}^{\text{periods}}\Big[ \mathbf{1}_{\{game[w,p,1]=i\wedge game[w,p,2]=j\}} + \mathbf{1}_{\{game[w,p,1]=j \wedge game[w,p,2]=i\}} \Big] = 1$

Where $\mathbf{1}_{{\cdot}}$ is the indicator function (1 if true, 0 if false).

In [3]:
%%writefile -a "./first_model_draft.mzn"

% 1
constraint
  forall(i, j in 1..n where i < j) (
    sum(w in 1..weeks, p in 1..periods) (
      bool2int( (game[w,p,1] == i /\ game[w,p,2] == j) \/
                (game[w,p,1] == j /\ game[w,p,2] == i) )
    ) == 1
  );

Appending to ./first_model_draft.mzn


<span style="color: blue;">
Explanation of the code above:

For every week w and every period p:

- Check if team i plays team j in that slot, in either order (team i in slot 1 and j in slot 2, or vice versa).

- `bool2int(...)` converts true/false to 1/0.

- The `sum` counts how many times teams i and j play each other across all weeks and periods.

This ensures that each pair of teams plays exactly once.
</span>

<span style="color: black;">
Basing on the literature <span style="color: purple;">@globalCs_SRRT</span> this is a Single Round Robin Tournament Problem (SRRT).

_"In round robin sport competitions, each team plays each other team a fixed number of times g during the competition. Let us first assume g 1⁄4 1, thus we are dealing with single round robin tournaments (SRR)."_
</span>

**2. every team plays once a week;**

For every week $w$ if you pick any two different periods the teams assigned to all the slots in these periods must be **different**. In math terms we can express this as:

$\forall w \in {1,\ldots,\text{weeks}}, \forall (p_1,s_1) \ne (p_2,s_2): game[w,p_1,s_1] \ne game[w,p_2,s_2].$


In [4]:
%%writefile -a "./first_model_draft.mzn"

% [2]
constraint 
  forall(w in 1..weeks)(
    alldifferent([game[w,p,s] | p in 1..periods, s in 1..slots])
);

Appending to ./first_model_draft.mzn


**3. every team plays at most twice in the same period over the tournament.**

$\forall i \in {1,\ldots,n},\ \forall p \in {1,\ldots,\text{periods}}: \quad \sum_{w=1}^{\text{weeks}} \sum_{s=1}^{\text{slots}} \begin{cases} 1, & \text{if } game[w,p,s] = i, \\ 0, & \text{if } game[w,p,s] \ne i \end{cases} \le 2$

(for every team (i) and every period (p), across all weeks and slots the team (i) may appear at most twice in the same period)


<span style="color: green;">SUGGESTION: try to use **count_gec** global constraint</span>
```minizinc
% Each team plays at most twice in the same period
constraint forall(t in Teams, p in 1..num_periods)(count_geq([period[t,j] | j in Teams where t != j], p, 2)::domain_propagation);
```


In [5]:
%%writefile -a "./first_model_draft.mzn"

% [3]
constraint forall(p in 1..periods) (
    global_cardinality_low_up(
        [game[w,p,s] | w in 1..weeks, s in 1..2],
        [i | i in 0..n-1],
        [0 | i in 0..n-1],  % lower bound: 0 appearances
        [2 | i in 0..n-1]   % upper bound: 2 appearances
    )
);


Appending to ./first_model_draft.mzn


## 4. Implied Constraints

**4.1.** The number of team must be even as the project requirements corresponds with what's written in the paper <span style="color: purple;">@globalCs_SRRT</span>: 

_"If n is even, the number of rounds is n 1. A DSRR with an odd number of teams consists of n rounds in each of which n 1 teams play and one team does not."_ 

So we could add a constraint to stop the solution of the problem if the number of teams is not even with:
```minizinc
constraint n mod 2 = 0;
```
However, in this context is probably not useful.

**4.2. Each game must be between two different teams**

$\forall w \in Weeks, \forall p \in Periods: \quad game[w,p,1] \neq game[w,p,2]$

This prevents a team from playing against itself in any slot.


In [6]:
%%writefile -a "./first_model_draft.mzn"

constraint
  forall(w in 1..weeks, p in 1..periods) (
    game[w,p,1] != game[w,p,2]
  );

Appending to ./first_model_draft.mzn


## 5. Symmetry Breaking Constraints

**5.1. Fix first week's first period slot**

By fixing the first game of the first week we break a part of the symmetry and reduce equivalent solutions from permuting teams.

In [7]:
%%writefile -a "./first_model_draft.mzn"

% Symmetry Breaking 1
constraint game[1, 1, 1] == 1;

Appending to ./first_model_draft.mzn


**5.2.Ordering weeks by home teams index in first period**

$For \ weeks \ w=1,…,n−2 \ \ \texttt{game}[w, 1, 1] < \texttt{game}[w+1, 1, 1]$

<span style="color: red;">PROBLEM:</span> this conflicts with **5.1** for n = 6, making it impossible to find a solution 

In [8]:
%%writefile -a "./first_model_draft.mzn"

% Symmetry Breaking 2
constraint forall(w in 1..weeks-1) (  game[w, 1, 1] < game[w+1, 1, 1]);

Appending to ./first_model_draft.mzn


**5.3. Break team pair symmetry** 

Every match (game) is a pair of teams. This pairs suffers from a simmetry problem, as each game can be expressed both as **team 1 vs team 2** and as **team 2 vs team 1**.
This simmetry can lead to $(n)!$ different solutions. So breaking it can definitily improve model speed.
To break the symmetry I decided to implement the following constraint: _ordering teams within a single match so that the home team index is lower than the away team index_

For any week $w$, period $p$, $\texttt{game}[w,p,1] < \texttt{game}[w,p,2]$ 

(This constraint is valid only for the decisional model. Instead, in the optimization model, we do care about team order as the number of home and away games for each team should be balanced)

In [9]:
%%writefile -a "./first_model_draft.mzn"

% Symmetry Breaking 3
constraint forall(w in 1..weeks, p in 1..periods) (  game[w,p,1] < game[w,p,2]);

Appending to ./first_model_draft.mzn


Solving the problem:

In [10]:
%%writefile -a "./first_model_draft.mzn"

solve satisfy;

Appending to ./first_model_draft.mzn


Python code to compare code execution results for different team numbers

<span style="color: red;">PROBLEM:</span> This code actually does not account for the limit of 300s for each problem instance.

In [16]:
import subprocess
import re
from tabulate import tabulate  # pip install tabulate

# Collect results

def compare_results(file_path, values, title, all_solutions=True):
    results = []

    for n in values:

        cmd = [
                "minizinc",
                file_path,
                "-D", f"n={n}",
                "--statistics",
                "--time-limit", "300000"
            ]

        if all_solutions:
            cmd.append("--all-solutions")

        # Run MiniZinc with all solutions and statistics
        result = subprocess.run(
            cmd,
            capture_output=True, text=True
        )

        
            

        output = result.stdout

        # ---- Parse output ----
        # Count number of solutions (each ends with "----------")
        num_solutions = output.count("----------")

        # Extract number of failures
        match_failures = re.search(r"%%%mzn-stat: failures=(\d+)", output)
        failures = int(match_failures.group(1)) if match_failures else None

        # Extract solver time
        match_time = re.search(r"%%%mzn-stat: solveTime=([0-9.]+)", output)
        solver_time = float(match_time.group(1)) if match_time else None

        # Add to results list
        results.append([n, num_solutions, failures, solver_time])

    # ---- Print results as a cute table ----
    headers = ["n", "Solutions", "Failures", "Solver Time (s)"]
    table = tabulate(results, headers=headers, floatfmt=".3f", tablefmt="fancy_grid")
    print(f"\n📊 {title}:")
    print(table)

In [17]:
values = [2, 4, 6, 8, 10, 12, 14]
compare_results("first_model_draft.mzn", values, "First draft" , all_solutions=False)


📊 First draft:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│   2 │           1 │          0 │             4.940 │
├─────┼─────────────┼────────────┼───────────────────┤
│   4 │           1 │          0 │             0.000 │
├─────┼─────────────┼────────────┼───────────────────┤
│   6 │           1 │          2 │             0.000 │
├─────┼─────────────┼────────────┼───────────────────┤
│   8 │           1 │         40 │             0.002 │
├─────┼─────────────┼────────────┼───────────────────┤
│  10 │           1 │       7358 │             0.257 │
├─────┼─────────────┼────────────┼───────────────────┤
│  12 │           1 │       1300 │             0.083 │
├─────┼─────────────┼────────────┼───────────────────┤
│  14 │           1 │      21925 │             1.455 │
╘═════╧═════════════╧════════════╧═══════════════════╛


## 6. Validation

The model must be implemented in MiniZinc and run using at least Gecode. Bonus points will be considered if other solvers are tried. The purpose of the validation is to assess the performance of the solvers using various models and search strategies. It is a good practice to evaluate also the contribution of the implied and symmetry breaking constraints.

### Experimental Design

Before presenting your experimental results, give all the details of your experimental study. Your results should be reproducible following your description. Explain which solvers you used and which search strategies you employed, as well as your experimental set up (e.g., the hardware and the software used, any posed time limit etc). Irreproducible experiments will not be considered.

### Experimental Results

Present your experimental results in a clear way. It is mandatory to show a table where:  
- rows are labeled with the identifier of the instances,
- columns are labeled with the different approaches you tried,  
- cells contain the runtime in seconds (in the decision version) or the best objective value (in the optimization version), as specified in the project description, found by the given approach on the given instance, using a certain search strategy. If the instance is solved to optimality, emphasize the objective value in bold. If the instance is proved to be unsatisfiable, indicate it as ’UNSAT’. If no answer is obtained within the time limit, indicate it with a ’N/A’ or ’-’. For example, “The results obtained using the search strategy h are reported in Table 1”.

## 9. Questions
- Is there a minimum number of teams for which our model should give a Solution? (currently 2, but no solution for 4 and 6) Is this because there are some conflicting constraints?

- Are there other possible simmetry breaking constraints?

- Can we use more global constraints?

- How to implement the optimization function in this model? Remember to use two separate files!
